In [ ]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
os.chdir("TensorFlowTTS")
!pip install .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10679, done.
remote: Total 10679 (delta 0), reused 0 (delta 0), pack-reused 10679
Receiving objects: 100% (10679/10679), 133.52 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (5163/5163), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/TensorFlowTTS
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 489.6 MB 22 kB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
     |████████████████████████████████| 235 kB 95.6 MB/s 
     |████████████████████████████████| 2

In [ ]:
!pip install git+https://github.com/repodiac/german_transliterate.git
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/repodiac/german_transliterate.git to /tmp/pip-install-cbq8wpph/german-transliterate_62e3a2aab9344bfd84d5e3217deb97ce
  Running command git clone -q https://github.com/repodiac/german_transliterate.git /tmp/pip-install-cbq8wpph/german-transliterate_62e3a2aab9344bfd84d5e3217deb97ce
     |████████████████████████████████| 125 kB 15.2 MB/s 
  Created wheel for german-transliterate: filename=german_transliterate-0.1.3-py3-none-any.whl size=20829 sha256=942f6df4b32b1828e6855e91a73ef1a6fb6d852dfa12123fe91e9f607f5feb0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0uutz1_9/wheels/77/17/55/6c6d2d33bd2b3b8a3741e12b17f0b18278861f64858bbcc228
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=92c1e2ff1921be2189d8c58334761ae3b6ea5a67c115e1af4b056e766ff183a4
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe

In [ ]:
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
tacotron2 = TFAutoModel.from_pretrained("tensorspeech/tts-tacotron2-kss-ko", name="tacotron2")

Downloading:   0%|          | 0.00/128M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

In [ ]:
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-kss-ko", name="fastspeech2")

Downloading:   0%|          | 0.00/125M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

In [ ]:
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-kss-ko", name="mb_melgan")

Downloading:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.82k [00:00<?, ?B/s]

In [ ]:
processor = AutoProcessor.from_pretrained("tensorspeech/tts-tacotron2-kss-ko")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MB-MELGAN":
    audio = vocoder_model.inference(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
  plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
  plt.show()
  plt.close()

In [ ]:

tacotron2.setup_window(win_front=10, win_back=10)

In [ ]:
input_texts = ["어서오세요", "너 그렇게 하면 군생활 많이 힘들어", "정신 똑바로 차려봐", "그렇게 하면 내가 진짜 실력이 없는 거 뭐 운도 없고 실력도 없고", "와서 방 가세요"]

In [ ]:
audio_= []

for input_text in input_texts: 

  mels, alignment_history, audios = do_synthesis(input_text, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
  ipd.Audio(audios, rate=22050)
  audio_.append(audios)


In [ ]:
audio_

[array([-0.00054912, -0.00067316, -0.00056447, ...,  0.00243555,
         0.00052332, -0.00086847], dtype=float32),
 array([ 2.7451080e-05, -1.9108507e-05,  5.2819119e-05, ...,
         7.4822619e-04, -4.1252503e-04, -1.1177326e-03], dtype=float32),
 array([-6.7522255e-05, -2.2674090e-04, -1.5278204e-04, ...,
         1.9712981e-03,  1.1981898e-04, -1.1143126e-03], dtype=float32),
 array([ 7.0991140e-05,  1.3569208e-04,  1.7442254e-04, ...,
         9.8957673e-05, -6.8893994e-04, -1.1389111e-03], dtype=float32),
 array([ 4.1534306e-05, -2.3856698e-04, -4.6352216e-04, ...,
         9.9882518e-04, -2.8652491e-04, -1.2980255e-03], dtype=float32)]

In [ ]:
ipd.Audio(audio_[0], rate=22050), ipd.Audio(audio_[1], rate=22050), ipd.Audio(audio_[2], rate=22050), ipd.Audio(audio_[3], rate=22050), ipd.Audio(audio_[4], rate=22050)

(<IPython.lib.display.Audio object>,
 <IPython.lib.display.Audio object>,
 <IPython.lib.display.Audio object>,
 <IPython.lib.display.Audio object>,
 <IPython.lib.display.Audio object>)

In [ ]:
from IPython.core.display import display
for i in range(5):
  display(ipd.Audio(audio_[i], rate=22050))

In [ ]:
ipd.Audio(audio_[0], rate=22050)

In [ ]:
ipd.Audio(audio_[1], rate=22050)

In [ ]:
ipd.Audio(audio_[2], rate=22050)

In [ ]:
ipd.Audio(audio_[3], rate=22050)

In [ ]:
ipd.Audio(audio_[4], rate=22050)

In [ ]:
sr = 21000

In [ ]:
path = "/content"

In [ ]:
import soundfile as sf
for i in range(5):
  sf.write(f'{path}/KsponSpeech_{str(i + 1).zfill(6)}.wav', audio_[i], sr)

In [ ]:
input_text = "어서오세요"

mels, alignment_history, audios = do_synthesis(input_text, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
ipd.Audio(audios, rate=22050)

In [ ]:
input_text = "너 그렇게 하면 군생활 많이 힘들어"

mels, alignment_history, audios = do_synthesis(input_text, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
ipd.Audio(audios, rate=22050)